In [1]:
from Classes.LoadModel import BaseModel
from Classes import Predict
# import memory_profiler

from Classes.Predict import analyze_face, Prediction
from sklearn.metrics import accuracy_score
from config import *
import random
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Layer, Input, Dropout, Activation, Convolution2D, MaxPooling2D


/home/ubuntu/anaconda3/envs/tensorflow2_latest_p37/lib/python3.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [2]:
MODEL = 0
LABEL = 1

In [3]:
def top_layers_vggface1(model_base):
    model_base.trainable = False
    model = Sequential()
    model.add(model_base)
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.25))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    return model

In [4]:
def top_layers_vggface7(model_base):
    model_base.trainable = False
    model = Sequential()
    model.add(model_base)
    model.add(Flatten())
    model.add(Dense(256,activation='relu'))
    model.add(Dense(128,activation='relu'))
    model.add(Dense(64,activation='relu'))
    model.add(Dense(5,activation='softmax'))
    return model

In [5]:
def load_best_model(model_name, label):
    # Loading BaseModel
    if model_name in ['emotion', 'gender', 'age', 'race']:
        basemodel = BaseModel(model_name)
    else:
        basemodel = BaseModel(model_name[:-1])
    model = basemodel.load_model(False)
    if label.lower() not in ['race', 'gender', 'emotion', 'age', 'hair_color'] :
        model = top_layers_vggface1(model)
        print('vggface1 layers loaded')
    elif 'hair' in label.lower():
        model = top_layers_vggface7(model)
        print('vggface7 layers loaded')
    else:
        model = model 
        print(att, 'model loaded')
    # Loading Best Model
    model.load_weights(os.path.join(MOD_ATT_PATH, f'{model_name}_{label}.h5'))
    print(f"\nBest Model {model_name}'s Arc. and Weights Loaded!")
    return model

In [6]:
# Taking Best Model per Att

IMAGE_PATH = '/home/ubuntu/sheryl/face_att_sheryl'
IND_FILE = '/home/ubuntu/jonathan/files_list.csv'
WEIGHT_PATH = '/home/ubuntu/sheryl/Facial_Attributes_Detection/Weights'
MOD_ATT_PATH =  '/home/ubuntu/sheryl/Facial_Attributes_Detection/model-att'

models_list = [file for file in os.listdir(MOD_ATT_PATH) if str(file).endswith('h5')]
best_model_list = []
label_list = []
best_pairs = []
# modelz = '/Users/tal/Dropbox/Projects/vggface_Eyeglasses.h5'

for model in models_list:
    best_model = model.split('/')[-1].split('_')[MODEL]
    label = ''.join(model.strip('.h5').split('_')[LABEL:])
    best_model_list.append(best_model)
    label_list.append(label)

best_pairs = zip(best_model_list, label_list)

print('Running Inference...')
label_query = ['Eyeglasses']


def inference(image_path, best_pairs, plot=False):

    # Get img list
    img_list = os.listdir(IMAGE_PATH)
    file = ''.join(random.choices(img_list, k=1))
    result = []
    score = []
    tic = time()
    file = os.path.join(image_path, file)

    for model_name, label in best_pairs:
        print(model_name)
        pos, neg = f'{label}: V', f'{label}: X'
        model = load_best_model(model_name, label)

        # running rage models
        result_rage, score_rage = analyze_face(file)
        # running binaryCls models
        bicls_result, bicls_score = Predict.predict_file(model, file, pos, neg)
        # running multiCls models ** I changed only here the label dict onmain it stays the same
        # labels_hair = {0: 'Bald', 1: 'Black_Hair', 2: 'Blond_Hair', 3: 'Brown_Hair', 4: 'Gray_Hair'}
        # result.append(Prediction.predict_label_multi(model, labels_hair, file, 'ResNet50'))

        result.append(bicls_result+result_rage)
        file_dict = {file: {label: score}}

        # update dictionary
        json_file = PATH_JSON + 'sum_file.json'
        with open(json_file, 'r') as jf:
            json_obj = json.load(jf)
        json_obj[file] = file_dict
        json_obj[file].update(score_rage)
        json_obj[file][label] = bicls_score[0][0].astype(float)

        with open(json_file, "w") as jf_out:
            json.dump(json_obj, jf_out)

    toc = time()
    run = toc - tic
    print(f'Avg Time inference {(run / 60)/len(models_list):.2f} minutes.')

    if plot:
        result = ''.join(result)
        img = mpimg.imread(file)
        plt.figure(figsize=(8, 5))
        plt.imshow(img)
        plt.text(s=result, x=190, y=100)
        plt.xticks([])
        plt.yticks([])
        plt.show()

    return file_dict


result = inference(IMAGE_PATH, best_pairs)
print(result)

# sum_json =
# print(sum_json)

Running Inference...
vggface1
vggface1 layers loaded

Best Model vggface1's Arc. and Weights Loaded!
age_model_weights.h5 will be downloaded...


Downloading...
From: https://drive.google.com/uc?id=1YCox_4kJ-BYeXq27uUbasu--yz28zUMV
To: /home/ubuntu/.deepface/weights/age_model_weights.h5
539MB [00:04, 121MB/s]  


gender_model_weights.h5 will be downloaded...


Downloading...
From: https://drive.google.com/uc?id=1wUXRVlbsni2FN9-jkS_f4UTUrm1bRLyk
To: /home/ubuntu/.deepface/weights/gender_model_weights.h5
537MB [00:08, 59.7MB/s] 


race_model_single_batch.h5 will be downloaded...


Downloading...
From: https://drive.google.com/uc?id=1nz-WDhghGQBC4biwShQ9kYjvQMpO6smj
To: /home/ubuntu/.deepface/weights/race_model_single_batch.zip
511MB [00:05, 85.7MB/s] 
Action: race: 100%|██████████| 4/4 [00:03<00:00,  1.14it/s]   


Face could not be detected


ValueError: not enough values to unpack (expected 2, got 0)